# LFP processing of Indefinite Streaming

In [258]:
# Importing Python and external packages


import os
import sys
import importlib
from importlib import reload 
from dataclasses import dataclass, field, fields
from itertools import compress
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler

import scipy
from scipy import stats
from scipy import signal
from scipy.signal import spectrogram, butter, filtfilt, freqz
from scipy.signal.windows import hann

import seaborn as sns
import pingouin as pg
from itertools import combinations
#from statannotations.Annotator import Annotator

import openpyxl
from openpyxl import Workbook, load_workbook
# import xlrd
import pickle
import json
import csv
import fooof
from fooof.plts.spectra import plot_spectrum

#mne
import mne
from mne.time_frequency import tfr_morlet 

# xdf
import pyxdf
import pyxdftools
from pyxdftools import XdfData

import datetime

In [162]:
%matplotlib qt


In [324]:
jennifer_user_path = os.getcwd()
while jennifer_user_path[-14:] != 'jenniferbehnke':
    jennifer_user_path = os.path.dirname(jennifer_user_path)

# directory to this Repository
project_path = os.path.join(jennifer_user_path, 'code', 'Patterned_stimulation_project', 'patterned_DBS')
sys.path.append(project_path)

os.chdir(project_path)


# data processing
import src.patterned_DBS.data_processing.indefinite_streaming as indef_streaming

# utility functions
import src.patterned_DBS.utils.io as io
import src.patterned_DBS.utils.find_folders as find_folders
import src.patterned_DBS.utils.lfp_preprocessing as lfp_preprocessing
import src.patterned_DBS.utils.xdf_structure as xdf_structure

importlib.reload(indef_streaming)
importlib.reload(io)
importlib.reload(find_folders)
importlib.reload(lfp_preprocessing)
importlib.reload(xdf_structure)

<module 'src.patterned_DBS.utils.xdf_structure' from '/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/xdf_structure.py'>

### 1. Load indefinite streaming (StimOFF), structure the data

In [327]:
indef_str_file = io.load_perceive_file(
    sub="080",
    modality="indefinite_streaming",
    task="rest",
    medication="on",
    stimulation="StimOffA",
    run="1",)

Creating RawArray with float64 data, n_channels=12, n_times=48851
    Range : 0 ... 48850 =      0.000 ...   195.400 secs
Ready.


/Users/jenniferbehnke/anaconda3/envs/burst_dbs/lib/python3.12/site-packages/pymatreader/utils.py:291: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn(
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:218: RuntimeWarning: The supplied FieldTrip structure does not have an elec or grad field. No channel locations will extracted and the kind of channel might be inaccurate.
  mne_data = mne.io.read_raw_fieldtrip(file_path, info={}, data_name="data")
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:218: RuntimeWarning: Cannot guess the correct type of channel LFP_Stn_L_03. Making it a MISC channel.
  mne_data = mne.io.read_raw_fieldtrip(file_path, info={}, data_name="data")
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:218: RuntimeWarning: Cannot 

In [150]:
# keys: "data", "mne_data", "ecg_cleaned_data", "ch_names", "file_path"]

indef_str_file["mne_data"].get_data().shape

(12, 54437)

In [149]:
indef_str_file["ecg_cleaned_data"].shape

(12, 54437)

In [152]:
indef_str_data = indef_streaming.structure_indef_str(
    sub="086",
    medication="on",
    stimulation="burst",
    run="1",
)

Creating RawArray with float64 data, n_channels=12, n_times=47538
    Range : 0 ... 47537 =      0.000 ...   190.148 secs
Ready.


/Users/jenniferbehnke/anaconda3/envs/burst_dbs/lib/python3.12/site-packages/pymatreader/utils.py:291: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn(
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:217: RuntimeWarning: The supplied FieldTrip structure does not have an elec or grad field. No channel locations will extracted and the kind of channel might be inaccurate.
  def load_metadata_excel(sub: str, sheet_name: str):
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:217: RuntimeWarning: Cannot guess the correct type of channel LFP_Stn_L_03. Making it a MISC channel.
  def load_metadata_excel(sub: str, sheet_name: str):
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:217: RuntimeWarning: Cannot guess the correct type of channel LFP_Stn_L_13

In [153]:
indef_str_data["data_info"]

,sub,medication,stimulation,run,fs,n_channels,n_samples_original,duration_original,n_samples_kept,duration_kept
0,086,on,burst,1,250.0,12,47538,190.152,45000,180.0


In [154]:
indef_str_data["raw_data"]

,LFP_Stn_L_03,LFP_Stn_L_13,LFP_Stn_L_02,LFP_Stn_R_03,LFP_Stn_R_13,LFP_Stn_R_02,LFP_Stn_L_01,LFP_Stn_L_12,LFP_Stn_L_23,LFP_Stn_R_01,LFP_Stn_R_12,LFP_Stn_R_23
0,-12.525555,-18.593126,-2.297396,-2.004049,-16.847570,4.635097,-6.067571,-8.364967,-10.228159,-14.843521,-10.208424,-6.639146
1,-11.971327,-16.255068,-4.375992,3.117410,-14.878373,7.614802,-4.283741,-8.659733,-7.595335,-17.995783,-10.380981,-4.497392
2,-4.766362,-10.465592,-0.328199,9.129559,-5.907589,10.594507,-5.699230,-6.027430,-4.438162,-15.037148,-4.442641,-1.464948
3,-1.884375,0.000000,1.969196,9.463567,0.000000,11.256664,1.884375,3.853572,-3.853572,-9.463567,1.793097,-1.793097
4,-1.662684,-0.668017,3.063194,6.123485,0.328199,9.049475,0.994668,4.057862,-4.725879,-5.795285,3.254190,-2.925990
...,...,...,...,...,...,...,...,...,...,...,...,...
44995,14.077394,17.368430,14.659574,2.338058,-2.187996,14.677807,3.291036,17.950610,-0.582180,-4.526054,10.151753,-12.339749
44996,13.301475,19.038471,15.097173,-0.445344,-0.546999,9.821991,5.736996,20.834169,-1.795698,-0.101655,9.720336,-10.267335
44997,8.978495,15.253044,8.533185,0.000000,1.969196,7.504443,6.274549,14.807734,0.445310,1.969196,9.473639,-7.504443
44998,6.650737,10.465592,7.220387,4.564779,8.533185,9.601273,3.814855,11.035242,-0.569650,3.968406,13.569679,-5.036494


In [155]:
indef_str_data["ecg_cleaned_data"]

,LFP_Stn_L_03,LFP_Stn_L_13,LFP_Stn_L_02,LFP_Stn_R_03,LFP_Stn_R_13,LFP_Stn_R_02,LFP_Stn_L_01,LFP_Stn_L_12,LFP_Stn_L_23,LFP_Stn_R_01,LFP_Stn_R_12,LFP_Stn_R_23
0,-12.525555,-18.593126,-2.297396,-2.004049,-16.847570,4.635097,-6.067571,-8.364967,-10.228159,-14.843521,-10.208424,-6.639146
1,-11.971327,-16.255068,-4.375992,3.117410,-14.878373,7.614802,-4.283741,-8.659733,-7.595335,-17.995783,-10.380981,-4.497392
2,-4.766362,-10.465592,-0.328199,9.129559,-5.907589,10.594507,-5.699230,-6.027430,-4.438162,-15.037148,-4.442641,-1.464948
3,-1.884375,0.000000,1.969196,9.463567,0.000000,11.256664,1.884375,3.853572,-3.853572,-9.463567,1.793097,-1.793097
4,-1.662684,-0.668017,3.063194,6.123485,0.328199,9.049475,0.994668,4.057862,-4.725879,-5.795285,3.254190,-2.925990
...,...,...,...,...,...,...,...,...,...,...,...,...
44995,14.077394,17.368430,4.594792,2.338058,-2.187996,14.677807,3.291036,6.633252,-0.582180,-4.526054,10.151753,0.104500
44996,13.301475,19.038471,1.859797,-0.445344,-0.546999,9.821991,5.736996,11.382932,-1.795698,-0.101655,9.720336,-2.215979
44997,8.978495,15.253044,4.704192,0.000000,1.969196,7.504443,6.274549,11.648204,0.445310,1.969196,9.473639,-1.428813
44998,6.650737,10.465592,4.485392,4.564779,8.533185,9.601273,3.814855,10.239431,-0.569650,3.968406,13.569679,1.416117


### 2. Only pick the channels around the stimulation contacts (02 or 13)

In [224]:
picked_channels = indef_streaming.pick_channel(
    sub="084",
    medication="on",    
    stimulation="burst",
    run="1",
)

Excel file loaded:  metadata_084.xlsx 
loaded from:  /Users/jenniferbehnke/OneDrive - Charité - Universitätsmedizin Berlin/Burst_DBS_project - AG Bewegungsstörungen/data/sub-084
Creating RawArray with float64 data, n_channels=12, n_times=49125
    Range : 0 ... 49124 =      0.000 ...   196.496 secs
Ready.


/Users/jenniferbehnke/anaconda3/envs/burst_dbs/lib/python3.12/site-packages/pymatreader/utils.py:291: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn(
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:217: RuntimeWarning: The supplied FieldTrip structure does not have an elec or grad field. No channel locations will extracted and the kind of channel might be inaccurate.
  mne_data = mne.io.read_raw_fieldtrip(file_path, info={}, data_name="data")
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:217: RuntimeWarning: Cannot guess the correct type of channel LFP_Stn_L_03. Making it a MISC channel.
  mne_data = mne.io.read_raw_fieldtrip(file_path, info={}, data_name="data")
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:217: RuntimeWarning: Cannot 

In [220]:
picked_channels["ecg_cleaned_data"].filter(like="Left")

,Left_02
0,-14.761681
1,-17.221960
2,-17.669285
3,-13.531540
4,-10.959430
...,...
44995,18.340270
44996,11.854076
44997,11.183091
44998,8.163656


In [183]:
picked_channels["ecg_cleaned_data"].shape[1]

2

In [19]:
picked_channels["data_info"]

,sub,medication,stimulation,run,fs,n_channels,n_samples_original,duration_original,n_samples_kept,duration_kept
0,084,on,burst,1,250.0,12,49125,196.5,45000,180.0


### 3. Plot raw and ecg cleaned time series

In [330]:
subjects = ["075", "080", "084", "086", "087", "088", "089"]

for sub in subjects:

    for stim in ["burst", "continuous"]:

        for run in ["1", "2", "3"]:

            plot_raw_time_series = indef_streaming.plot_time_series(
                sub=sub,
                medication="on",    
                stimulation=stim,
                run=run,
                channels="around_stim",
                raw_or_ecg_cleaned="ecg_cleaned",
            )

In [331]:
subjects = ["075", "080", "084", "086", "087", "088", "089"]

for sub in subjects:

    plot_tfr = indef_streaming.plot_time_frequency(
        sub=sub,
        medication="on",    
        raw_or_ecg_cleaned="ecg_cleaned",
    )

# no ecg_cleaned in data sub-075 and 080
# sub-086 metadata_excel ValueError stream is empty

Excel file loaded successfully: metadata_075.xlsx 
loaded from: /Users/jenniferbehnke/Dropbox/work/ResearchProjects/Patterned_stimulation_project/data/sub-075
Creating RawArray with float64 data, n_channels=12, n_times=48813
    Range : 0 ... 48812 =      0.000 ...   195.248 secs
Ready.


/Users/jenniferbehnke/anaconda3/envs/burst_dbs/lib/python3.12/site-packages/pymatreader/utils.py:291: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn(
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:218: RuntimeWarning: The supplied FieldTrip structure does not have an elec or grad field. No channel locations will extracted and the kind of channel might be inaccurate.
  mne_data = mne.io.read_raw_fieldtrip(file_path, info={}, data_name="data")
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:218: RuntimeWarning: Cannot guess the correct type of channel LFP_Stn_L_03. Making it a MISC channel.
  mne_data = mne.io.read_raw_fieldtrip(file_path, info={}, data_name="data")
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:218: RuntimeWarning: Cannot 

Excel file loaded successfully: metadata_075.xlsx 
loaded from: /Users/jenniferbehnke/Dropbox/work/ResearchProjects/Patterned_stimulation_project/data/sub-075
Creating RawArray with float64 data, n_channels=12, n_times=48813
    Range : 0 ... 48812 =      0.000 ...   195.248 secs
Ready.


/Users/jenniferbehnke/anaconda3/envs/burst_dbs/lib/python3.12/site-packages/pymatreader/utils.py:291: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn(
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:218: RuntimeWarning: The supplied FieldTrip structure does not have an elec or grad field. No channel locations will extracted and the kind of channel might be inaccurate.
  mne_data = mne.io.read_raw_fieldtrip(file_path, info={}, data_name="data")
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:218: RuntimeWarning: Cannot guess the correct type of channel LFP_Stn_L_03. Making it a MISC channel.
  mne_data = mne.io.read_raw_fieldtrip(file_path, info={}, data_name="data")
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:218: RuntimeWarning: Cannot 

No data available
Excel file loaded successfully: metadata_075.xlsx 
loaded from: /Users/jenniferbehnke/Dropbox/work/ResearchProjects/Patterned_stimulation_project/data/sub-075
Creating RawArray with float64 data, n_channels=12, n_times=48875
    Range : 0 ... 48874 =      0.000 ...   195.496 secs
Ready.


...

Excel file loaded successfully: metadata_089.xlsx 
loaded from: /Users/jenniferbehnke/Dropbox/work/ResearchProjects/Patterned_stimulation_project/data/sub-089
Creating RawArray with float64 data, n_channels=12, n_times=46975
    Range : 0 ... 46974 =      0.000 ...   187.896 secs
Ready.


/Users/jenniferbehnke/anaconda3/envs/burst_dbs/lib/python3.12/site-packages/pymatreader/utils.py:291: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn(
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:218: RuntimeWarning: The supplied FieldTrip structure does not have an elec or grad field. No channel locations will extracted and the kind of channel might be inaccurate.
  mne_data = mne.io.read_raw_fieldtrip(file_path, info={}, data_name="data")
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:218: RuntimeWarning: Cannot guess the correct type of channel LFP_Stn_L_03. Making it a MISC channel.
  mne_data = mne.io.read_raw_fieldtrip(file_path, info={}, data_name="data")
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:218: RuntimeWarning: Cannot 

Excel file loaded successfully: metadata_089.xlsx 
loaded from: /Users/jenniferbehnke/Dropbox/work/ResearchProjects/Patterned_stimulation_project/data/sub-089
Creating RawArray with float64 data, n_channels=12, n_times=46975
    Range : 0 ... 46974 =      0.000 ...   187.896 secs
Ready.


/Users/jenniferbehnke/anaconda3/envs/burst_dbs/lib/python3.12/site-packages/pymatreader/utils.py:291: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn(
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:218: RuntimeWarning: The supplied FieldTrip structure does not have an elec or grad field. No channel locations will extracted and the kind of channel might be inaccurate.
  mne_data = mne.io.read_raw_fieldtrip(file_path, info={}, data_name="data")
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:218: RuntimeWarning: Cannot guess the correct type of channel LFP_Stn_L_03. Making it a MISC channel.
  mne_data = mne.io.read_raw_fieldtrip(file_path, info={}, data_name="data")
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:218: RuntimeWarning: Cannot 

Figures sub-089_med-on_stim-OFF_time_frequency_IS_Right_ecg_cleaned.svg and sub-089_med-on_stim-OFF_time_frequency_IS_Right_ecg_cleaned.png 
were written in: /Users/jenniferbehnke/Dropbox/work/ResearchProjects/Patterned_stimulation_project/figures/sub-089.


In [247]:
plot_tfr

,sub,medication,stimulation,run,fs,n_channels,n_samples_original,duration_original,n_samples_kept,duration_kept
0,084,on,burst,1,250.0,12,49125,196.5,45000,180.0
